In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)
    if "cassavav16" in dirname:
        for f in filenames:
            print(f)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input
/kaggle/input/cassavav03
/kaggle/input/effb45thfold1
/kaggle/input/cassavav02
/kaggle/input/cassavav05
/kaggle/input/cassavav08
/kaggle/input/cassavav06
/kaggle/input/cassavav12
/kaggle/input/cassavav16
legacy_seresnext26_32x4d_fold0_best_epoch_29_final_4th.pth
legacy_seresnext26_32x4d_fold2_best_epoch_26_final_5th.pth
legacy_seresnext26_32x4d_fold1_best_epoch_21_final_5th.pth
legacy_seresnext26_32x4d_fold3_best_epoch_4_final_5th.pth
legacy_seresnext26_32x4d_fold4_best_epoch_17_final_5th.pth
/kaggle/input/cassavav11
/kaggle/input/cassavav13
/kaggle/input/cassavav14
/kaggle/input/cassavav07
/kaggle/input/cassavav01
/kaggle/input/cassavav01/utils
/kaggle/input/cassavav01/weights
/kaggle/input/cassavav01/pytorch-image-models
/kaggle/input/cassavav01/pytorch-image-models/tests
/kaggle/input/cassavav01/pytorch-image-models/results
/kaggle/input/cassavav01/pytorch-image-models/.github
/kaggle/input/cassavav01/pytorch-image-models/.github/ISSUE_TEMPLATE
/kaggle/input/cassavav01/

In [2]:
!pip install /kaggle/input/cassavav01/pytorch-image-models > /dev/null

In [3]:
!cp -r /kaggle/input/cassavav01/utils . 

In [4]:
from collections import defaultdict
import copy
import random
import numpy as np
import os
import shutil
from urllib.request import urlretrieve
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
cudnn.benchmark = True
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
import timm
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy, JsdCrossEntropy
from utils import Mixup, RandAugment
from PIL import Image
SEED = 42

In [5]:
def seed_everything(SEED):
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(SEED)

In [6]:
root = os.path.join(os.environ["HOME"], "/kaggle/input/cassava-leaf-disease-classification")
os.listdir(root)

['train_tfrecords',
 'sample_submission.csv',
 'test_tfrecords',
 'label_num_to_disease_map.json',
 'train_images',
 'train.csv',
 'test_images']

In [7]:
train = pd.read_csv(f'{root}/train.csv')
test = pd.read_csv(f'{root}/sample_submission.csv')
label_map = pd.read_json(f'{root}/label_num_to_disease_map.json', 
                         orient='index')
display(train.head())
display(test.head())
display(label_map)

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


,image_id,label
0,2216849948.jpg,4


,0
0,Cassava Bacterial Blight (CBB)
1,Cassava Brown Streak Disease (CBSD)
2,Cassava Green Mottle (CGM)
3,Cassava Mosaic Disease (CMD)
4,Healthy


In [8]:
models_name = ["resnest26d", "resnest50d" ,"tf_efficientnet_b3_ns", "tf_efficientnet_b4_ns", "legacy_seresnext26_32x4d","vit_base_patch16_384"]


WEIGHTS = [
#     "../input/cassavav02/resnest26d_fold0_best_epoch_4_final_512.pth",
#     "../input/cassavav01/weights/resnest26d_fold1_best_epoch_7_finale_ex_hnm.pth",
#     "../input/cassavav02/resnest26d_fold2_best_epoch_4_final_512.pth",
#     "../input/cassavav02/resnest26d_fold4_best_epoch_21_final_512.pth",
    #
#     "../input/cassavav05/resnest26d_fold0_best_epoch_19_final_3rd.pth",
#     "../input/cassavav05/resnest26d_fold1_best_epoch_7_final_2nd.pth",
#     "../input/cassavav05/resnest26d_fold2_best_epoch_4_final_2nd.pth",
#     "../input/cassavav05/resnest26d_fold3_best_epoch_10_final_3rd.pth",
#     "../input/cassavav05/resnest26d_fold4_best_epoch_6_final_3rd.pth",
    
#     "../input/cassavav03/tf_efficientnet_b3_ns_fold1_best_epoch_1_final_512.pth",
    #
#     "../input/cassavav06/resnest50d_fold0_best_epoch_13_final_2nd.pth",
#     "../input/cassavav07/resnest50d_fold0_best_epoch_10_final_3rd.pth",
#     "../input/cassavav08/resnest50d_fold0_best_epoch_16_final_4th.pth",
#     "../input/cassavav06/resnest50d_fold1_best_epoch_17_final_2nd.pth",
#     "../input/cassavav09/resnest50d_fold1_best_epoch_8_final_5th_pseudo.pth",
#     "../input/cassavav06/resnest50d_fold2_best_epoch_22_final_2nd.pth",
#     "../input/cassavav06/resnest50d_fold3_best_epoch_2_final_2nd.pth",
#     "../input/cassavav07/resnest50d_fold3_best_epoch_1_final_3rd.pth",
#     "../input/cassavav08/resnest50d_fold3_best_epoch_2_final_4th.pth",
#     "../input/cassavav06/resnest50d_fold4_best_epoch_10_final_2nd.pth",
#     "../input/cassavav06/resnest50d_fold4_best_epoch_15_final_3rd.pth",
#     "../input/cassavav08/resnest50d_fold4_best_epoch_10_final-4th.pth",
#     "../input/cassavav09/resnest50d_fold4_best_epoch_1_final_5th_pseudo.pth",
    
#     "../input/cassavav11/tf_efficientnet_b4_ns_fold0_best_epoch_3_final_3rd.pth",
#     "../input/cassavav11/tf_efficientnet_b4_ns_fold1_best_epoch_16_final_3rd.pth",
#     "../input/cassavav11/tf_efficientnet_b4_ns_fold2_best_epoch_5_final_3rd.pth",
    "../input/cassavav11/tf_efficientnet_b4_ns_fold4_best_epoch_20_final_3rd.pth",
    
#     "/kaggle/input/cassavav12/tf_efficientnet_b4_ns_fold0_best_epoch_25_final_3rd.pth",
    "/kaggle/input/cassavav12/tf_efficientnet_b4_ns_fold0_best_epoch_14_final_2nd_loss.pth",
#     "/kaggle/input/cassavav12/tf_efficientnet_b4_ns_fold0_best_epoch_24_final_2nd.pth",
#     "/kaggle/input/cassavav12/tf_efficientnet_b4_ns_fold2_best_epoch_7_final_3rd.pth",
    "../input/effb43rdfold2/tf_efficientnet_b4_ns_fold2_best_epoch_29_final_3rd.pth",
# #     "/kaggle/input/cassavav12/tf_efficientnet_b4_ns_fold2_best_epoch_20_final_2nd.pth",
#     "/kaggle/input/cassavav12/tf_efficientnet_b4_ns_fold1_best_epoch_30_final_3rd.pth",
# #     "/kaggle/input/cassavav12/tf_efficientnet_b4_ns_fold1_best_epoch_20_final_2nd.pth",
#     "/kaggle/input/cassavav12/tf_efficientnet_b4_ns_fold3_best_epoch_16_final_3rd.pth",
# #     "/kaggle/input/cassavav12/tf_efficientnet_b4_ns_fold3_best_epoch_27_final_2nd.pth",
#     "/kaggle/input/cassavav12/tf_efficientnet_b4_ns_fold4_best_epoch_20_final_3rd.pth",
#     "/kaggle/input/cassavav12/tf_efficientnet_b4_ns_fold4_best_epoch_23_final_2nd.pth",
    
    "../input/effb45thfold1/tf_efficientnet_b4_ns_fold1_best_epoch_26_final_5th.pth",
    
#     "/kaggle/input/cassavav13/tf_efficientnet_b4_ns_fold0_best_epoch_14_final_2nd.pth",
#     "/kaggle/input/cassavav13/tf_efficientnet_b4_ns_fold1_best_epoch_7_final_2nd.pth",
#     "/kaggle/input/cassavav13/tf_efficientnet_b4_ns_fold1_best_epoch_10_final_2nd_acc.pth",
#     "/kaggle/input/cassavav13/tf_efficientnet_b4_ns_fold2_best_epoch_11_final_2nd_acc.pth",
#     "/kaggle/input/cassavav13/tf_efficientnet_b4_ns_fold2_best_epoch_1_final_2nd.pth",
    "/kaggle/input/cassavav13/tf_efficientnet_b4_ns_fold3_best_epoch_14_final_2nd.pth",
#     "/kaggle/input/cassavav13/tf_efficientnet_b4_ns_fold4_best_epoch_13_final_2nd_acc.pth",
#     "/kaggle/input/cassavav13/tf_efficientnet_b4_ns_fold4_best_epoch_5_final_2nd.pth",
 
# "/kaggle/input/cassavav14/legacy_seresnext26_32x4d_fold0_best_epoch_22_final_2nd.pth",
# "/kaggle/input/cassavav14/legacy_seresnext26_32x4d_fold1_best_epoch_1_final_fmix_3rd.pth",
# "/kaggle/input/cassavav14/legacy_seresnext26_32x4d_fold1_best_epoch_32_final_2nd.pth "
# "/kaggle/input/cassavav14/legacy_seresnext26_32x4d_fold1_best_epoch_30_final_3rd.pth",
# "/kaggle/input/cassavav14/legacy_seresnext26_32x4d_fold2_best_epoch_28_final_2nd.pth",
# "/kaggle/input/cassavav14/legacy_seresnext26_32x4d_fold3_best_epoch_28_final_2nd.pth",
# "/kaggle/input/cassavav14/legacy_seresnext26_32x4d_fold4_best_epoch_22_final_2nd.pth",
# "/kaggle/input/cassavav14/legacy_seresnext26_32x4d_fold4_best_epoch_16_final_3rd.pth",

#     "/kaggle/input/cassavav15/legacy_seresnext26_32x4d_fold0_best_epoch_29_final_4th.pth",
#     "/kaggle/input/cassavav15/legacy_seresnext26_32x4d_fold1_best_epoch_17_final_4th.pth",
#     "/kaggle/input/cassavav15/legacy_seresnext26_32x4d_fold2_best_epoch_28_final_4th.pth",
#     "/kaggle/input/cassavav15/legacy_seresnext26_32x4d_fold3_best_epoch_14_final_4th.pth",
#     "/kaggle/input/cassavav15/legacy_seresnext26_32x4d_fold4_best_epoch_27_final_4th.pth",
#     "/kaggle/input/cassavav16/legacy_seresnext26_32x4d_fold0_best_epoch_29_final_4th.pth",
#     "/kaggle/input/cassavav16/legacy_seresnext26_32x4d_fold2_best_epoch_26_final_5th.pth",
#     "/kaggle/input/cassavav16/legacy_seresnext26_32x4d_fold1_best_epoch_21_final_5th.pth",
#     "/kaggle/input/cassavav16/legacy_seresnext26_32x4d_fold3_best_epoch_4_final_5th.pth",
#     "/kaggle/input/cassavav16/legacy_seresnext26_32x4d_fold4_best_epoch_17_final_5th.pth",

    
]
model_index = 3
ckpt_index = 0

ensemble_models_name = list(8*["tf_efficientnet_b4_ns"])
ensemble_ckpt_index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

params = {
    "visualize": False,
    "debug":False,
    "fold": [0,1,2,3,4],
    "load_pretrained": True,
    "image_size": 512,
    "num_classes": 5,
    "model": models_name[model_index],
    "device": "cuda",
    "batch_size": 32,
    "num_workers": 2,
    "drop_block": 0.2,
    "drop_rate": 0.2,
    "tta": True,
    "kfold_pred":True,
    "error_fix":False,
    "ensemble":False
}

In [9]:
class TestDataset(Dataset):
    def __init__(self, df, transform=None, valid_test=False):
        self.df = df
        self.file_names = df['image_id'].values
        self.transform = transform
        self.valid_test = valid_test
        if self.valid_test:
            self.labels = df['label'].values  
        else:
            assert ValueError("Test data does not have annotation, plz check!")
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{root}/test_images/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if isinstance(self.transform, list):
            outputs = {'images':[],
                       'labels':[]}
             #image0 = transforms.ToPILImage()(image)
             #image0 = self.transform[0](image0)

            for trans in self.transform:
                augmented = trans(image=image)
                image_aug = augmented['image']
                outputs["images"].append(image_aug)
                del image_aug
                
            if self.valid_test:
                label = torch.tensor(self.labels[idx]).long()
                outputs['labels'] = len(self.transform)*[label]
            else:
                outputs['labels'] = len(self.transform)*[-1]
                
            return outputs
        else:
            augmented = self.transform(image=image)
            image = augmented['image'] 
        return image

In [10]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor

In [11]:
def declare_model(name, index=None):
    if "efficientnet" in name:
        model = timm.create_model(
                params["model"],
                pretrained=False,
                num_classes=params["num_classes"], 
                drop_rate=params["drop_rate"], 
                drop_path_rate=0.2)
    elif "seresnext" in name:
        model = timm.create_model(name,
                pretrained=False,
                num_classes=params["num_classes"])
    else:
        model = timm.create_model(name,
                pretrained=False,
                num_classes=params["num_classes"],
                drop_rate=params["drop_rate"])
        
    model = model.to(params["device"])
    model = torch.nn.DataParallel(model) 
        
    if params["load_pretrained"]:
        if index == None: 
            return model
        state_dict = torch.load(WEIGHTS[index])
        print(f"Load pretrained model: {name} ",state_dict["preds"])
        model.load_state_dict(state_dict["model"])
        best_acc = state_dict["preds"]   
    return model
if not params["kfold_pred"] and not params["ensemble"]:
    model = declare_model(params["model"], ckpt_index)

In [12]:
transform_tta0 = A.Compose(
    [
     A.CenterCrop(height=params["image_size"], width=params["image_size"], p=1),
     A.Resize(height=params["image_size"], width=params["image_size"], p=1),
     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),   
     ToTensorV2()
    ]
)

transform_tta1 = A.Compose(
    [
     A.CenterCrop(height=params["image_size"], width=params["image_size"], p=1),
     A.Resize(height=params["image_size"], width=params["image_size"], p=1),
     A.HorizontalFlip(p=1.),
     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),   
     ToTensorV2()
    ]
)
transform_tta2 = A.Compose(
    [
     A.CenterCrop(height=params["image_size"], width=params["image_size"], p=1),        
     A.Resize(height=params["image_size"], width=params["image_size"], p=1),
     A.VerticalFlip(p=1.),
     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
     ToTensorV2(),
    ]
)
transform_tta3 = A.Compose(
    [
     A.CenterCrop(height=params["image_size"], width=params["image_size"], p=1),
     A.Resize(height=params["image_size"], width=params["image_size"], p=1),
     A.RandomRotate90(p=1.),
     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),     
     ToTensorV2(),
    ]
)
##### Test TTA with Five Crops
transform_crop_tta0 = T.Compose([
                T.FiveCrop(params["image_size"]),
                T.Lambda(lambda crops: ([T.ToTensor()(crop) for crop in crops])),
                T.Lambda(lambda norms: torch.stack([T.Normalize(mean=[0.5], std=[0.5])(norm) for norm in norms]))
        ]
)
test_transform_tta = [transform_tta0, transform_tta1, transform_tta2, transform_tta3]

#debug
if params['debug']:
    print("In debug mode")
    test = pd.concat([test,test, test])

if params["tta"]:
    test_pred_dataset = TestDataset(test, transform=test_transform_tta)
else:
    test_pred_dataset = TestDataset(test, transform=test_transform_tta[0])
    
test_pred_loader = DataLoader(
    test_pred_dataset, batch_size=params["batch_size"], shuffle=False, num_workers=params['num_workers'], pin_memory=True,
)

In [13]:
test.head

<bound method NDFrame.head of          image_id  label
0  2216849948.jpg      4>

In [14]:
def visualize_tta(data, pred):     
    unorm = UnNormalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    if params["tta"]:
        figure, ax = plt.subplots(nrows=1, ncols=num_tta, figsize=(12, 6))
        for i, image in enumerate(data["images"]):
            image = (unorm(image[0]).cpu().numpy()*255).astype(int)
            ax.ravel()[i].imshow(image.transpose(2,1,0))
            ax.ravel()[i].set_title(str(pred), color='GREEN')
            ax.ravel()[i].set_axis_off()
        plt.tight_layout()
        plt.show() 
    else:
        image = (unorm(data).cpu().numpy()*255).astype(int)
        imgplot = plt.imshow(image[0].transpose(2,1,0))
        imgplot = plt.title(str(pred), color='GREEN')
        plt.show()

In [15]:
def gmean(input_x, dim):
    log_x = torch.log(input_x)
    return torch.exp(torch.mean(log_x, dim=dim))

In [16]:
num_tta = len(test_transform_tta)
def tta_predict(loader, params, valid_test=False):
    outputs = []
    preds = []
    models = []
    for i, ckpt_idx in enumerate(params["fold"]):
        model = declare_model(params["model"], i)
        model.eval()
        models.append(model)
    #    
    stream = tqdm(loader)    
    with torch.no_grad():
        for i, data in enumerate(stream, start=1):
            if params["visualize"]:
                visualize_tta(data, pred)
            tta_output = []   
            for i, image in enumerate(data["images"]):
                kfout = [torch.softmax(model(image), dim=1) for model in models]
                tta_output.extend(torch.stack(kfout, dim=0))
            tta_output = torch.stack(tta_output, dim=0).mean(dim=0)
            
            pred = torch.argmax(tta_output, dim=1).cpu().numpy()    
            if params["error_fix"]:
                topk_output, topk_ids = torch.topk(output, params["num_classes"])
                for i in range(len(pred)):
                    ## adjust the output prediction
                    max_1st = topk_ids[i][0]
                    max_2nd = topk_ids[i][1]
                    if  max_1st == 0 and max_2nd == 4 and output[i][max_2nd] > 0.2:
                        pred[i] = max_2nd
                    if max_1st == 3 and max_2nd == 2 and output[i][max_2nd] > 0.2:
                        pred[i] = max_2nd

            preds.extend(pred)
            ## For visualize the TTA 
            if params["visualize"]:
                visualize_tta(data, pred)
    return preds

In [17]:
# Ensemble CV w/ fold predict on validation set
def tta_pred_ensemble_model(loader, params, valid_test=False):
    stream = tqdm(loader)
    models = []
    preds = []
    print(f"Ensemble below models: {ensemble_models_name}")
    for name, ckpt in zip(ensemble_models_name, ensemble_ckpt_index):
        print(ckpt)
        m = declare_model(name, ckpt)  
        models.append(m.eval())
        del m
    
    with torch.no_grad():
        for i, data in enumerate(stream, start=1):
            tta_output = []   
            for i, image in enumerate(data["images"]):
                kfout = []
                for model,name in zip(models, ensemble_models_name):
                    if "eb6" in name or "sr_101" in name:                    
                        with torch.cuda.amp.autocast(), torch.no_grad():
                            image = image.cuda()
                            kfout.append(torch.softmax(model(image), dim=1))
                    else:
                        kfout.append(torch.softmax(model(image), dim=1))
                        
                tta_output.extend(torch.stack(kfout, dim=0))
            tta_output = torch.stack(tta_output, dim=0).mean(dim=0)
            
#             if params["stacking"]:
#                 tta_output = F.softmax(tta_output, 1).data.cpu().numpy()
#                 preds.append(tta_output)

            pred = torch.argmax(tta_output, dim=1).cpu().numpy() 
            preds.extend(pred)
    
    ## For visualize the TTA 
        if params["visualize"]:
            visualize_tta(data, pred)
    return preds     
if params["ensemble"]:
    final_out = tta_pred_ensemble_model(test_pred_loader, params)  
else:
    final_out = tta_predict(test_pred_loader, params)             

Load pretrained model: tf_efficientnet_b4_ns  0.8945
Load pretrained model: tf_efficientnet_b4_ns  0.8879
Load pretrained model: tf_efficientnet_b4_ns  0.8902
Load pretrained model: tf_efficientnet_b4_ns  0.8972


  0%|          | 0/1 [00:00<?, ?it/s]

Load pretrained model: tf_efficientnet_b4_ns  0.8922


100%|██████████| 1/1 [00:02<00:00,  2.54s/it]


In [18]:
test['label'] = final_out 
test.to_csv('submission.csv', index=False)
test.head(n=10)

,image_id,label
0,2216849948.jpg,2
